d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Gain Actionable Insights from Twitter Data

In this capstone project, you will use Structured Streaming to gain insight from streaming Twitter data.

The executive team would like to have access to some key business metrics such as
* most tweeted hashtag in last 5 minute window
* a map of where tweets are coming from

## Instructions
* Insert solutions wherever it says `FILL_IN`
* Feel free to copy/paste code from the previous notebooks, where applicable
* Run test cells to verify that your solution is correct

## Audience
* Primary Audience: Data Engineers
* Additional Audiences: Data Analysts and Data Scientists

## Prerequisites
* Web browser: current versions of Google Chrome, Firefox, Safari, Microsoft Edge and 
Internet Explorer 11 on Windows 7, 8, or 10 (see <a href="https://docs.databricks.com/user-guide/supported-browsers.html#supported-browsers#" target="_blank">Supported Web Browsers</a>)
* Databricks Runtime 4.2 or greater
* Completed courses Spark-SQL, DataFrames or ETL-Part 1 from <a href="https://academy.databricks.com/" target="_blank">Databricks Academy</a>, or have similar knowledge
* Have done the rest of this course

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Getting Started</h2>

Run the following cell to configure our "classroom."

In [4]:
%run "./Includes/Classroom-Setup"

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 1</h2>
<h3>Read Streaming Data from Input Source</h3>

The input source is a a Kafka feed of Twitter data

For this step you will need to:
0. Use the `format()` operation to specify "kafka" as the type of the stream
0. Specify the location of the Kafka server by setting the option "kafka.bootstrap.servers" with one of the following values (depending on where you are located): 
 * **server1.databricks.training:9092** (US-Oregon)
 * **server2.databricks.training:9092** (Singapore)
0. Indicate which topics to listen to by setting the option "subscribe" to "tweets"
0. Throttle Kafka's processing of the streams
0. Rewind stream to beginning when we restart notebook
0. Load the input data stream in as a DataFrame
0. Select the column `value` - cast it to a `STRING`

In [6]:
from pyspark.sql.functions import col

spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

kafkaServer = "server1.databricks.training:9092"   # US (Oregon)
# kafkaServer = "server2.databricks.training:9092" # Singapore

rawDF = (spark.readStream
 .format("kafka")                     # Specify "kafka" as the type of the stream
 .option("kafka.bootstrap.servers",kafkaServer)                     # Set the location of the kafka server
 .option("subscribe","tweets")                     # Indicate which topics to listen to
 .option("maxOffsetsPerTrigger",1000)                     # Throttle Kafka's processing of the streams
 .option("startingOffsets","earliest")                     # Rewind stream to beginning when we restart notebook
 .load()                     # Load the input data stream in as a DataFrame
 .select(col("value").cast("STRING"))                     # Select the "value" column and cast to a string
)

In [7]:
# TEST - Run this cell to test your solution.
schemaStr = str(rawDF.schema)

dbTest("SS-06-schema-value",     True, "(value,StringType,true)" in schemaStr)
dbTest("SS-06-is-streaming",     True, rawDF.isStreaming)

print("Tests passed!")

Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 2</h2>
<h3>A Schema for parsing JSON</h3>

Becase the schema is so complex, it is being provided for you.

Simply run the following cell and proceed to the next step.

In [9]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, ArrayType

twitSchema = StructType([
  StructField("hashTags", ArrayType(StringType(), False), True),
  StructField("text", StringType(), True),   
  StructField("userScreenName", StringType(), True),
  StructField("id", LongType(), True),
  StructField("createdAt", LongType(), True),
  StructField("retweetCount", IntegerType(), True),
  StructField("lang", StringType(), True),
  StructField("favoriteCount", IntegerType(), True),
  StructField("user", StringType(), True),
  StructField("place", StructType([
    StructField("coordinates", StringType(), True), 
    StructField("name", StringType(), True),
    StructField("placeType", StringType(), True),
    StructField("fullName", StringType(), True),
    StructField("countryCode", StringType(), True)]), 
  True)
])

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 3</h2>
<h3>Create a JSON DataFrame</h3>

From the `rawDF` parse out the json subfields using `from_json`. Create a DataFrame that has fields
* `time`
* `json`, a nested field that has all the rest of the data
* promote all `json` subfields to fields.

In [11]:
from pyspark.sql.functions import from_json, expr

cleanDF = (rawDF
 .withColumn("json", from_json(col("value"), twitSchema))                          # Add the column "json" by parsing the column "value" with "from_json"
 .select(
   expr("cast(cast(json.createdAt as double)/1000 as timestamp) as time"),   # Cast "createdAt" column properly, call it "time"
   col("json.hashTags").alias("hashTags"),                                     # Promote subfields of "json" column e.g. "json.field" to "field"
   col("json.text").alias("text"),
   col("json.userScreenName").alias("userScreenName"),
   col("json.id").alias("id"),
   col("json.retweetCount").alias("retweetCount"),
   col("json.lang").alias("lang"),
   col("json.favoriteCount").alias("favoriteCount"),
   col("json.user").alias("user"),
   col("json.place.coordinates").alias("coordinates"), 
   col("json.place.name").alias("name"),
   col("json.place.placeType").alias("placeType"),
   col("json.place.fullName").alias("fullName"),
   col("json.place.countryCode").alias("countryCode")#Repeat for each subfields of "json"
 )
)

In [12]:
# TEST - Run this cell to test your solution.
schemaStr = str(cleanDF.schema)

dbTest("SS-06-schema-hashTag",  True, "hashTags,ArrayType(StringType,true)" in schemaStr)
dbTest("SS-06-schema-text",  True, "(text,StringType,true)" in schemaStr)
dbTest("SS-06-schema-userScreenName",  True, "(userScreenName,StringType,true)" in schemaStr)
dbTest("SS-06-schema-id",  True, "(id,LongType,true)" in schemaStr)
dbTest("SS-06-schema-time",  True, "(time,TimestampType,true)" in schemaStr)
dbTest("SS-06-schema-retweetCount",  True, "(retweetCount,IntegerType,true)" in schemaStr)
dbTest("SS-06-schema-lang",  True, "(lang,StringType,true)" in schemaStr)
dbTest("SS-06-schema-favoriteCount",  True, "(favoriteCount,IntegerType,true)" in schemaStr)
dbTest("SS-06-schema-user",  True, "(user,StringType,true)" in schemaStr)
dbTest("SS-06-schema-coordinates",  True, "(coordinates,StringType,true)" in schemaStr)
dbTest("SS-06-schema-name",  True, "(name,StringType,true)" in schemaStr)
dbTest("SS-06-schema-placeType",  True, "(placeType,StringType,true)" in schemaStr)
dbTest("SS-06-schema-fullName",  True, "(fullName,StringType,true)" in schemaStr)
dbTest("SS-06-schema-countryCode",  True, "(countryCode,StringType,true)" in schemaStr)

print("Tests passed!")

Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 4</h2>
<h3>Display Twitter Data as a Table</h3>

Click the left-most button in the bottom left corner.

In [14]:
display(cleanDF)  # display "cleanDF"

time,hashTags,text,userScreenName,id,retweetCount,lang,favoriteCount,user,coordinates,name,placeType,fullName,countryCode
2020-09-13T06:47:21.000+0000,List(),I be thinking back like sheesh,BabyMilo718,1305035321390903296,0,en,0,Yonder🗽,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(),I want to eat them and i still didn't sleep,_bunbooks_,1305035321411883009,0,en,0,BunBooks,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(),RT @lowkeyalbert: You glow different when you pray.,wnizzahh,1305035321407545345,0,en,0,ℤ𝕒𝕙𝕙𝕙,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(),RT @Bill_Nye_Tho: hey wear your mask correctly,Cardenassss456,1305035321386528769,0,en,0,Chris Cardenas,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(),"RT @MyHustlethread: Pink or sky blue which one you rocking,Dm if you’re looking to buy dem wristwatches and also go through my media for on…",iamorjiemmanuel,1305035321390911489,0,en,0,👑Ceo,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(img20plus),"RT @BIGRUDEFUCK: 𝖗𝖙,𝖉𝖒 #img20plus",daboyfongfux,1305035321403346944,0,en,0,ฟองฟุ,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(),RT @dairyfreejuuls: you know its a bad day when u wake up,Andrews25687395,1305035321403469824,0,en,0,Andrews,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(),"RT @julianne_dk: 'Thankfully we are getting closer tgt. We can talk casually as friends... right?' 'Aren't we friends bfr filming?' 'Ofc, w…",HyunJi3230,1305035321399074817,0,en,0,Seo Yea Ji stan,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(),RT @VISUALSLEGEND: late nights. https://t.co/JFMVhsVHnj,eurphorica,1305035321415856128,0,en,0,nana.,[],null,null,null,null
2020-09-13T06:47:21.000+0000,List(),@HeLuvsLi_ 🍯🍯🍯freaks 21+ follow me I follow back freaky tweeters only 🍯🍯🍯,DaSexperience,1305035321390829574,0,en,0,Kitty Kat,[],null,null,null,null


In [15]:
# TEST - Run this cell to test your solution.
dbTest("SS-06-numActiveStreams", True, len(spark.streams.active) > 0)
       
print("Tests passed!")

Tests passed!

When you are done, stop the stream:

In [17]:
for s in spark.streams.active:
  s.stop()

In [18]:
# TEST - Run this cell to test your solution.
dbTest("SS-06-numActiveStreams1", 0, len(spark.streams.active))

print("Tests passed!")

Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 5</h2>
<h3>Hashtag Processing</h3>

In this exercise, we do ETL processing on the `hashTags` column.

The goal is to first convert hash tags all to lower case then group tweets and count by hash tags.

You will notice that `hashTags` is an array of hash tags, which you will have to break up (use `explode` function).

The `explode` method allows you to split an array column into multiple rows, copying all the other columns into each new row.

In [20]:
from pyspark.sql.functions import explode, lower

twitCountsDF = (cleanDF      # Start with "cleanDF"
 .select(explode(col("hashTags")).alias("hashTag"),# Explode the array "hashTags" into "hashTag"
         lower(col("hashTag")))      # Convert "hashTag" to lower case                               
 .groupBy(col("hashTag"))                    # Aggregate by "hashTag"...
 .count()                    # For the aggregate, produce a count  
 .orderBy(col("count"))                    # Sort by "count"
 .limit(25)                    # Limit the result to 25 records
)

In [21]:
# TEST - Run this cell to test your solution.
schemaStr = str(twitCountsDF.schema)

dbTest("SS-06-schema-hashTag", True, "(hashTag,StringType,true)" in schemaStr)
dbTest("SS-06-schema-count",   True, "(count,LongType,false)" in schemaStr)

print("Tests passed!")

Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 6</h2>
<h3>Plot Counts of Top 25 Most Popular Hashtags</h3>

Under <b>Plot Options</b>, use the following:
* <b>Keys:</b> `hashTag`
* <b>Values:</b> `count`

In <b>Display type</b>, use <b>Pie Chart</b> and click <b>Apply</b>.

Once you apply the plot options, be prepared to increase the size of the plot graphic using the resize widget in the lower right corner of the graphic area.

In [23]:
display(twitCountsDF) # display twitCountsDF

hashTag,count
LetsStart22ndwithSAVE,1
COURSES,1
Bellator246,1
sex,1
bungomayoi,1
1BillionViews,1
sriramya,1
FollowDileepAddicts,1
원우,1
IPCLive,1


In [24]:
# TEST - Run this cell to test your solution.
dbTest("SS-06-numActiveStreams", True, len(spark.streams.active) > 0)
       
print("Tests passed!")

Tests passed!

When you are done, stop the stream:

In [26]:
for streamingQuery in spark.streams.active:
  streamingQuery.stop()

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 7</h2>
<h3>Read in File with Two Letter to Three Letter Country Codes</h3>

For this next part we are going to take a look at the number of requests per country.

To get started, we first need a lookup table that will give us the 3-character country code.

0. Read in the file at `/mnt/training/countries/ISOCountryCodes/ISOCountryLookup.parquet`
0. We will be interested in the `alpha2Code` and `alpha3Code` fields later

In [28]:
countryCodeDF = spark.read.parquet("/mnt/training/countries/ISOCountryCodes/ISOCountryLookup.parquet")

In [29]:
# TEST - Run this cell to test your solution.
schemaStr = str(countryCodeDF.schema)

dbTest("SS-06-schema-1", True, "(EnglishShortName,StringType,true)" in schemaStr)
dbTest("SS-06-schema-2", True, "(alpha2Code,StringType,true)" in schemaStr)
dbTest("SS-06-schema-3", True, "(alpha3Code,StringType,true)" in schemaStr)
dbTest("SS-06-schema-4", True, "(numericCode,StringType,true)" in schemaStr)
dbTest("SS-06-schema-5", True, "(ISO31662SubdivisionCode,StringType,true)" in schemaStr)
dbTest("SS-06-schema-6", True, "(independentTerritory,StringType,true)" in schemaStr)

dbTest("SS-06-streaming-7", False, countryCodeDF.isStreaming)

print("Tests passed!")

Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 8</h2>
<h3>Join Tables &amp; Aggregate By Country</h3>

In `cleanDF`, there is a `countryCode` field. However, it is in the form of a two-letter country code.

The `display` map expects a three-letter country code.

In order to retrieve tweets with three-letter country codes, we will have to join `cleanDF` with `countryCodesDF`.

In [31]:
from pyspark.sql.functions import isnull
mappedDF = (cleanDF
  .filter(~isnull(col("countryCode")))                            # Filter out any nulls for "countryCode"
  .join(countryCodeDF, col("countryCode") == col("alpha2Code"))  # Join the two tables on "countryCode" and "alpha2Code"
  .groupBy("alpha3Code")                            # Aggregate by country, "alpha3Code"
  .count()                            # Produce a count of each aggregate
)

In [32]:
# TEST - Run this cell to test your solution.
schemaStr = str(mappedDF.schema)
print(schemaStr)

dbTest("SS-06-schema-1",  True, "alpha3Code,StringType,true" in schemaStr)
dbTest("SS-06-schema-2",  True, "count,LongType,false" in schemaStr)

print("Tests passed!")

StructType(List(StructField(alpha3Code,StringType,true),StructField(count,LongType,false)))
Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 9</h2>
<h3>Plot Tweet Counts on a World Map</h3>

Under <b>Plot Options</b>, use the following:
* <b>Keys:</b> `alpha3Code`
* <b>Values:</b> `count`

In <b>Display type</b>, use <b>World map</b> and click <b>Apply</b>.

<img src="https://files.training.databricks.com/images/eLearning/Structured-Streaming/plot-options-map-06.png"/>

In [34]:
display(mappedDF)  # display mappedDF

alpha3Code,count
IND,20
MWI,1
NZL,2
TUR,1
LKA,2
OMN,1
DEU,1
AUS,6
NGA,9
PHL,7


In [35]:
# TEST - Run this cell to test your solution.
dbTest("SS-06-numActiveStreams", True, len(spark.streams.active) > 0)
       
print("Tests passed!")

Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 10: Write Stream</h2>

Write the stream to an in-memory table
0. Use appropriate `format`
0. For this exercise, we want to append new records to the results table
0. Gives the query a name
0. Start the query
0. Assign the query to `mappedTable`

In [37]:
mappedQuery = (mappedDF 
.writeStream                               # From the DataFrame get the DataStreamWriter
.format("memory")                               # Specify the sink format as "memory"
.outputMode("complete")                               # Configure the output mode as "complete"
.queryName("mappedTablePython")                               # Name the query "mappedTablePython"
.start()                               # Start the query
)

In [38]:
# TEST  - Run this cell to test your solution.
dbTest("SS-06-isActive", True, mappedQuery.isActive)
dbTest("SS-06-name", "mappedTablePython", mappedQuery.name)

print("Tests passed!")

Tests passed!

Wait until stream is done initializing...

In [40]:
untilStreamIsReady("mappedTablePython")

The stream is active and ready.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 11: Use SQL Syntax to Display a Few Rows</h2>

Do a basic SQL query to display all columns and, say, 10 rows.

### Why are we doing this?

In [42]:
%sql
SELECT *
FROM mappedTablePython
LIMIT 10

alpha3Code,count
IND,26
MWI,1
NZL,2
TUR,2
LKA,2
OMN,1
DEU,2
AUS,10
NGA,10
PHL,8


In [43]:
# TEST - Run this cell to test your solution.
try: tableExists = (spark.table("mappedTablePython") is not None)
except: tableExists = False
dbTest("SS-06-1", True, tableExists)  

firstRowCol = spark.sql("SELECT * FROM mappedTablePython limit 1").first()[0]
dbTest("SS-06-rowsExist", True, len(firstRowCol) > 0) 

print("Tests passed!")

Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Step 12: Stop Streaming Jobs</h2>

Before we can conclude, we need to shut down all active streams.

In [45]:
for s in spark.streams.active:  # Iterate over all the active streams
  s.stop()  # Stop the stream

In [46]:
# TEST - Run this cell to test your solution.
dbTest("SS-06-numActiveStreams", 0, len(spark.streams.active))

print("Tests passed!")

Tests passed!

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>